In [ ]:
__author__ = "Nagarajan"

## LRU Cache

Requirements:
 - cache should have size n
 - should keep the most recently accessed n items in the cache
 - when new item is accessed, the last item in cache is removed and new item is inserted.
 - cache can be increased or decreased at runtime (optional)
 


In [1]:


class LRUCache(object):
    """
    Implementation of an LRU cache using a doubly linked list
    and dict.
    It has O(1) lookups and is blazingly fast (for python).
    """
    def __init__(self, size, ds):
        """
        Setup the size and datastore.
        Instantiate the doubly linked list and nodeMap
        """
        self.size = size
        self.dlist = DoublyLinkedList()
        self.keyToNodeMap = {}
        self.ds = ds
        
    def get(self, key):
        """
        Return the data if its in the cache.
        Else, get it from the datastore, store it in the cache and 
        return the data.
        """
        if key in keyToNodeMap:
            # hit
            node = keyToNodeMap[key]
            self.dlist.moveNodeToHead(node)
        else:
            # miss
            value = self.ds.get(key)
            if self.dlist.getSize() >= self.size:
                self.dlist.removeLast()
            node = self.dlist.insertValue(value, 0)
            self.keyToNodeMap[key] = node
        return node.value
    
    def updateSize(self, size):
        """
        """
        if size < self.size:
            diff = self.size - size
            for x in xrange(diff):
                node = self.dlist.removeLast()
                del self.keyToNodeMap[node.key]
        self.size = size


class DoublyLinkedList(object):
    pass 

            
        
            
        

In [7]:
a = {1:2, 3:4}
del a[3]
print a

{1: 2}


Pretty cool `'x'`